In [6]:
import pandas as pd
import numpy as np
import sklearn
from xgboost import XGBClassifier
import xgboost
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pickle
from sklearn.utils import shuffle

In [7]:
df = pd.read_csv('../data/train.csv', delimiter=",", usecols = ['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv', 'click_bool', 'gross_bookings_usd',
       'booking_bool'])
df = shuffle(df)

Lets train it on clicking

In [6]:
trainOnlyColumns = ['position','click_bool','booking_bool','gross_bookings_usd']
Y = df['click_bool']
df.drop(trainOnlyColumns, axis=1,inplace=True)

In [7]:
n = 20
numOfRowsTest = (int(len(df)*(n/100)))
X_test = df.tail(numOfRowsTest)
Y_test = Y[-numOfRowsTest:]
df.drop(df.tail(numOfRowsTest).index,inplace=True)
y_train = Y[:(len(Y)-numOfRowsTest)]

In [19]:
# construct xgboost.DMatrix from numpy array, treat -999.0 as missing value
xgmat = xgboost.DMatrix( df, label=y_train)

# setup parameters for xgboost
param = {}
# use logistic regression loss, use raw prediction before logistic transformation
# since we only need the rank
param['objective'] = 'reg:logistic'
# scale weight of positive examples
param['scale_pos_weight'] = sum(y_train==0)/sum(y_train==1)
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'
param['silent'] = 0
param['nthread'] = 10

# you can directly throw param in, though we want to watch multiple metrics here
plst = list(param.items())+[('eval_metric', 'ams@0.15')]

watchlist = [ (xgmat,'train') ]
# boost 120 trees
num_round = 120
print ('loading data end, start to boost trees')
bst = xgboost.train( plst, xgmat, num_round, watchlist );
# save out model
bst.save_model('paraTest.model')

print ('finish training')

# fit model no training data
#model = XGBClassifier()
#model.fit(xs, ys)
# make predictions for test data

E:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
E:\Anaconda\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


loading data end, start to boost trees
[0]	train-auc:0.702969	train-ams@0.15:91.6018
[1]	train-auc:0.714316	train-ams@0.15:94.9236
[2]	train-auc:0.720499	train-ams@0.15:97.1739
[3]	train-auc:0.724368	train-ams@0.15:98.4507
[4]	train-auc:0.727643	train-ams@0.15:99.8537
[5]	train-auc:0.731166	train-ams@0.15:101.068
[6]	train-auc:0.733877	train-ams@0.15:101.778
[7]	train-auc:0.736402	train-ams@0.15:102.591
[8]	train-auc:0.738269	train-ams@0.15:103.283
[9]	train-auc:0.740919	train-ams@0.15:104.19
[10]	train-auc:0.74345	train-ams@0.15:105.098
[11]	train-auc:0.746016	train-ams@0.15:105.754
[12]	train-auc:0.747944	train-ams@0.15:106.405
[13]	train-auc:0.750381	train-ams@0.15:107.336
[14]	train-auc:0.752429	train-ams@0.15:108.027
[15]	train-auc:0.754508	train-ams@0.15:108.658
[16]	train-auc:0.7567	train-ams@0.15:109.568
[17]	train-auc:0.758544	train-ams@0.15:110.239
[18]	train-auc:0.760362	train-ams@0.15:110.902
[19]	train-auc:0.762185	train-ams@0.15:111.476
[20]	train-auc:0.764248	train-ams@0

KeyboardInterrupt: 

In [ ]:
xgmat_test = xgboost.DMatrix( X_test, label=Y_test)
y_pred = bst.predict(xgmat_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
Counter(predictions)

Lets train it on booking

In [8]:
trainOnlyColumns = ['position','click_bool','booking_bool','gross_bookings_usd']
Y = df['booking_bool']
df.drop(trainOnlyColumns, axis=1,inplace=True)

In [9]:
n = 20
numOfRowsTest = (int(len(df)*(n/100)))
X_test = df.tail(numOfRowsTest)
Y_test = Y[-numOfRowsTest:]
df.drop(df.tail(numOfRowsTest).index,inplace=True)
y_train = Y[:(len(Y)-numOfRowsTest)]

In [ ]:
# construct xgboost.DMatrix from numpy array, treat -999.0 as missing value
xgmat = xgboost.DMatrix( df, label=y_train)

# setup parameters for xgboost
param = {}
# use logistic regression loss, use raw prediction before logistic transformation
# since we only need the rank
param['objective'] = 'reg:logistic'
# scale weight of positive examples
param['scale_pos_weight'] = sum(y_train==0)/sum(y_train==1)
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'
param['silent'] = 0
param['nthread'] = 10

# you can directly throw param in, though we want to watch multiple metrics here
plst = list(param.items())+[('eval_metric', 'ams@0.15')]

watchlist = [ (xgmat,'train') ]
# boost 120 trees
num_round = 25
print ('loading data end, start to boost trees')
bst = xgboost.train( plst, xgmat, num_round, watchlist , early_stopping_rounds=10);
# save out model
bst.save_model('bookingModel.model')

print ('finish training')

# fit model no training data
#model = XGBClassifier()
#model.fit(xs, ys)
# make predictions for test data

In [ ]:
xgmat_test = xgboost.DMatrix( X_test, label=Y_test)
y_pred = bst.predict(xgmat_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
Counter(predictions)

Lets make a submission

In [7]:
bst = xgboost.Booster({'nthread': 12})  # init model
bst.load_model('xgboostBinary26April.model')  # load data


In [ ]:
dfTest = pd.read_csv('../data/test.csv', delimiter=",", usecols = ['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price',  'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv'])
xgmat_test_submission = xgboost.DMatrix( dfTest)

In [ ]:
y_pred_clicking = bst.predict(xgmat_test_submission)


In [ ]:
bst = xgboost.Booster({'nthread': 12})  # init model
bst.load_model('bookingModel.model')  # load data

In [ ]:
y_pred_booking = bst.predict(xgmat_test_submission)

In [ ]:
y_pred = y_pred_clicking + y_pred_booking
dfTest['predictedPos'] = y_pred
dfTest = dfTest.sort_values(['srch_id','predictedPos'],ascending=[True, False])


In [ ]:
print(dfTest.head())
dfSubmission = dfTest[['srch_id','prop_id']]
dfSubmission.to_csv("submissionTest2.csv",index=False)

In [ ]:
#Counter(y_pred)
predictions = [round(value) for value in y_pred]
Counter(predictions)